In [1]:
import pandas as pd

# read in

In [2]:
df = pd.read_csv('../../2_Feature_Engineering/export/combined_polly_bretschneider_iwg.csv')
df.head()

,raw_text,severity,binary_label
0,ist ekelhaft bei ihren Klos kannst du nur mit...,0.0,False
1,Bevor Sie mit Jud...,1.0,True
2,Und de Amis freut es wenn die Heuschrecken Eur...,0.0,False
3,": Die Krim ist ein heißer Tip, auch zum In...",1.0,True
4,: Es muss in #Deutschland eine politische Kra...,1.0,True


# clean

In [3]:
import sys
sys.path.insert(0, r'..\..\0_common')
from model_helpers import clean_all as ci

In [4]:
df['cleaned_text'] = df['raw_text'].apply(ci)
df.head()

,raw_text,severity,binary_label,cleaned_text
0,ist ekelhaft bei ihren Klos kannst du nur mit...,0.0,False,ekelhaft klos kannst gummistiefel gehen stinkt...
1,Bevor Sie mit Jud...,1.0,True,bevor judenstern markiert worden juden ganze ...
2,Und de Amis freut es wenn die Heuschrecken Eur...,0.0,False,de amis freut heuschrecken europa klein wirtsc...
3,": Die Krim ist ein heißer Tip, auch zum In...",1.0,True,krim heißer tip investieren muß bürger landes...
4,: Es muss in #Deutschland eine politische Kra...,1.0,True,deutschland politische kraft geben sozial sch...


In [5]:
df.shape

(9169, 4)

# vectorize

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
vec = CountVectorizer(ngram_range=(1,3), analyzer='word', max_features=100000)

In [8]:
X_features = vec.fit_transform(df['cleaned_text'])

In [9]:
X_features.shape

(9169, 100000)

# initialize model

In [10]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=-1, n_estimators=100)

# Cross validation

In [11]:
from sklearn.model_selection import cross_validate
scores = cross_validate(clf, X_features, df['binary_label'], cv=5,
                        scoring=('f1', 'roc_auc', 'precision', 'recall'),
                        return_train_score=True)

In [12]:
sorted(scores.keys())

['fit_time',
 'score_time',
 'test_f1',
 'test_precision',
 'test_recall',
 'test_roc_auc',
 'train_f1',
 'train_precision',
 'train_recall',
 'train_roc_auc']

In [13]:
scores_df = pd.DataFrame(scores)
scores_df

,fit_time,score_time,test_f1,train_f1,test_roc_auc,train_roc_auc,test_precision,train_precision,test_recall,train_recall
0,49.012147,1.775163,0.871275,0.999710,0.955110,0.999893,0.963534,1.0,0.795139,0.999421
1,45.127280,1.267305,0.849315,0.999566,0.961807,0.999930,0.973094,1.0,0.753472,0.999132
2,43.896334,1.472729,0.820000,0.999710,0.951994,0.999896,0.966981,1.0,0.711806,0.999421
3,45.083155,1.573417,0.862069,0.999566,0.959115,0.999885,0.961538,1.0,0.781250,0.999132
4,46.484599,1.263619,0.861871,0.999711,0.965743,0.999893,0.956215,1.0,0.784473,0.999421


In [14]:
# metrics
scores_df[['test_f1', 'test_roc_auc', 'test_precision', 'test_recall']].mean()

test_f1           0.852906
test_roc_auc      0.958754
test_precision    0.964273
test_recall       0.765228
dtype: float64